import packages

In [ ]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

load training / testing dataset

In [ ]:
filepath = 'train.csv'
default_values = ['x', 0.0]
header = True

csv_dataset = tf.data.experimental.CsvDataset(filepath, default_values, header=header)

def reshape_dataset(txt, tag):
  txt = tf.reshape(txt, shape=(1,))
  tag = tf.reshape(tag, shape=(1,))
  return txt, tag

csv_dataset = csv_dataset.map(reshape_dataset)
csv_dataset

<MapDataset element_spec=(TensorSpec(shape=(1,), dtype=tf.string, name=None), TensorSpec(shape=(1,), dtype=tf.float32, name=None))>

load valiadtion dataset

In [ ]:
filepath = 'validation.csv'
default_values = ['x', 0.0]
header = True

val_dataset = tf.data.experimental.CsvDataset(filepath, default_values, header=header)

val_dataset = val_dataset.map(reshape_dataset)
val_dataset

<MapDataset element_spec=(TensorSpec(shape=(1,), dtype=tf.string, name=None), TensorSpec(shape=(1,), dtype=tf.float32, name=None))>

print comments and its label

In [ ]:
for entry in csv_dataset:
    print('text: {}, tag: {}'.format(entry[0], entry[1]))

In [ ]:
for entry in val_dataset:
    print('text: {}, tag: {}'.format(entry[0], entry[1]))

split training and testing datasets

In [ ]:
training = csv_dataset.take(4000)
testing = csv_dataset.skip(4000)

build the model

In [ ]:
import tensorflow_hub as hub
from tensorflow.keras.layers import Dropout

model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[], input_shape=[], 
                           dtype=tf.string, trainable=True)
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


**train the model**

EarlyStopping: [lnk](https://blog.csdn.net/qq_21466543/article/details/106843058)

In [ ]:
from keras.callbacks import EarlyStopping

model.compile(   optimizer = tf.keras.optimizers.Adam(learning_rate=0.002),
          loss=tf.losses.BinaryCrossentropy(from_logits=False),
          metrics=[tf.metrics.BinaryAccuracy(threshold=0.5, name='accuracy')])


early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

history = model.fit(csv_dataset,
                  epochs=50,
                  batch_size=128,
                  validation_data = val_dataset,
                  verbose=1,
                  callbacks=[early_stopping]
                    )

Epoch 1/50
4553/4553 [==============================] - 27s 6ms/step - loss: 0.9304 - accuracy: 0.9337 - val_loss: 3.6483 - val_accuracy: 0.7515
Epoch 2/50
4553/4553 [==============================] - 28s 6ms/step - loss: 0.8397 - accuracy: 0.9398 - val_loss: 3.7460 - val_accuracy: 0.7395
Epoch 3/50
4553/4553 [==============================] - 28s 6ms/step - loss: 0.8160 - accuracy: 0.9429 - val_loss: 3.4763 - val_accuracy: 0.7635
Epoch 4/50
4553/4553 [==============================] - 29s 6ms/step - loss: 0.8209 - accuracy: 0.9427 - val_loss: 3.4344 - val_accuracy: 0.7675
Epoch 5/50
4553/4553 [==============================] - 27s 6ms/step - loss: 0.7663 - accuracy: 0.9468 - val_loss: 3.4048 - val_accuracy: 0.7695
Epoch 6/50
4553/4553 [==============================] - 26s 6ms/step - loss: 0.7276 - accuracy: 0.9482 - val_loss: 3.4928 - val_accuracy: 0.7675
Epoch 7/50
4553/4553 [==============================] - 26s 6ms/step - loss: 0.7058 - accuracy: 0.9504 - val_loss: 3.4334 - val_ac

save the model

In [ ]:
model.save("model")

load trained model

In [ ]:
from tensorflow import keras
new_model = keras.models.load_model("model")

show the performance of model

In [ ]:
results = new_model.evaluate(testing)

print(results)

In [ ]:
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

Create a graph of accuracy and loss over time

In [ ]:
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

use the model to make predictions

In [ ]:
import numpy as np
import pandas as pd
import csv
import tensorflow as tf


#open csv file containing comments need to be predicted
filepath = 'comment_4.csv'
default_values = ['x', 0.0]
header = True

prediction_dataset = tf.data.experimental.CsvDataset(filepath, default_values, header=header)
prediction_dataset = prediction_dataset.map(reshape_dataset)
prediction_dataset

predictions = new_model.predict(prediction_dataset) 
print(predictions) 

#write prediction into a file
file = open('output.csv', 'w+', newline ='') 
with file:   
    write = csv.writer(file)
    write.writerows(predictions)